In [1]:
%load_ext autoreload
%autoreload 2
import os, pickle, csv, itertools, shutil, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision, torch
from scipy import stats

%matplotlib inline
%config InlineBackend.figure_format='retina'

import utils.plot_data as plot
import models.transforms as transforms
import algorithms.teaching_algs as algs
import evals.teaching_evals as evals

def get_ci(samples, confidence=0.95):  return 2 * stats.sem(samples) * stats.t.ppf((1 + confidence) / 2., len(samples)-1)
import warnings
warnings.filterwarnings("ignore")

In [8]:
dataset = "bm"
transform = transforms.shrink(63)
# transform = transforms.no_transform()

In [9]:
train = torchvision.datasets.ImageFolder(f"data/datasets/{dataset}/train", transform=transform)
test = torchvision.datasets.ImageFolder(f"data/datasets/{dataset}/test", transform=transform)
x_train = np.array([x[0].numpy() for x in train])
x_test = np.array([x[0].numpy() for x in test])
y_train = np.array([x[1] for x in train])
y_test = np.array([x[1] for x in test])

In [10]:
lpips_path = f"data/dist/lpips/{dataset}"
lpips_alex = pickle.load(open(f"{lpips_path}/lpips.alex.test.pkl","rb"))
lpips_vgg = pickle.load(open(f"{lpips_path}/lpips.vgg.test.pkl","rb"))
lpips_squeeze = pickle.load(open(f"{lpips_path}/lpips.squeeze.test.pkl","rb"))

In [12]:
X = np.array([x.flatten() for x in x_train])

### random examples

In [13]:
n_trials = 1000

random_scores = np.array([evals.get_knn_score_lpips(lpips_alex, algs.random(X, 10), y_train, y_test) for _ in range(n_trials)])
random_ci = get_ci(random_scores)
random_scores = random_scores.mean()
random_scores, random_ci

(0.6464249999999999, 0.013150072225918009)

In [14]:
n_trials = 1000

random_scores = np.array([evals.get_knn_score_lpips(lpips_vgg, algs.random(X, 10), y_train, y_test) for _ in range(n_trials)])
random_ci = get_ci(random_scores)
random_scores = random_scores.mean()
random_scores, random_ci

(0.6539749999999999, 0.01291163869049983)

### protodash

In [15]:
teaching_idx = algs.protodash(X, 10)
evals.get_knn_score_lpips(lpips_alex, teaching_idx, y_train, y_test, k=1), \
evals.get_knn_score_lpips(lpips_vgg, teaching_idx, y_train, y_test, k=1), \
evals.get_knn_score_lpips(lpips_squeeze, teaching_idx, y_train, y_test, k=1), \

(0.825, 0.725, 0.8)

### k-medoids

In [16]:
teaching_idx = algs.k_medoids(X, 10)
evals.get_knn_score_lpips(lpips_alex, teaching_idx, y_train, y_test, k=1), \
evals.get_knn_score_lpips(lpips_vgg, teaching_idx, y_train, y_test, k=1), \
evals.get_knn_score_lpips(lpips_squeeze, teaching_idx, y_train, y_test, k=1), \

(0.775, 0.775, 0.75)